# <center> **Coded Apertures: 2D Image Recostruction Test**<center>

**Libraries**

In [32]:
import builtins
import collections.abc as c

import numpy as np
from scipy.linalg import hankel
from scipy.signal import correlate

import plot_module

np.set_printoptions(precision=2, suppress=False)

**Correlation Function**

**Sky Background**

**Detector Image**

**Sky Reconstruction**

**Signal-to-Noise Ratio**

### **Random Mask Pattern**

### **URA Mask Pattern**

### **References**